In [1]:
def loadDataSet():
    return [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not {item} in C1:
                C1.append({item})
    C1.sort()
    return list(map(frozenset, C1))  #frozenset可以作为键，set不行
    
def scanD(D, Ck, minSupport):
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):  #判断是否是子集
                if not (can in ssCnt):  #键是否在字典里
                    ssCnt[can] = 1
                else:
                    ssCnt[can] += 1
    numItems = float(len(D))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key] / numItems
        if support >= minSupport:
            retList.insert(0, key)  #插在列表最前面
        supportData[key] = support
    return retList, supportData

In [2]:
dataSet = loadDataSet()
dataSet

[[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

In [3]:
C1 = createC1(dataSet)
C1

[frozenset({1}),
 frozenset({3}),
 frozenset({4}),
 frozenset({2}),
 frozenset({5})]

In [4]:
D = list(map(set, dataSet))
D

[{1, 3, 4}, {2, 3, 5}, {1, 2, 3, 5}, {2, 5}]

In [5]:
L1, suppData0 = scanD(D, C1, 0.5)
L1

[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})]

In [6]:
suppData0

{frozenset({1}): 0.5,
 frozenset({3}): 0.75,
 frozenset({4}): 0.25,
 frozenset({2}): 0.75,
 frozenset({5}): 0.75}

## 发现频繁项集

In [7]:
def aprioriGen(Lk, k):
    '''
    输入为频繁项集列表Lk和下一步要生成的项集里元素个数k
    输出为Ck
    '''
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk):
            L1 = list(Lk[i])[:k-2]  #确保没有重复值
            L2 = list(Lk[j])[:k-2]
            L1.sort()
            L2.sort()
            if L1 == L2:
                retList.append(Lk[i] | Lk[j])
    return retList

def apriori(dataSet, minSupport=0.5):
    C1 = createC1(dataSet)
    D = list(map(set, dataSet))
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)
        supportData.update(supK)  #更新
        L.append(Lk)
        k += 1
    return L, supportData

In [8]:
L, suppData = apriori(dataSet)
L

[[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})],
 [frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3})],
 [frozenset({2, 3, 5})],
 []]

In [9]:
aprioriGen(L[0], 2)

[frozenset({2, 5}),
 frozenset({3, 5}),
 frozenset({1, 5}),
 frozenset({2, 3}),
 frozenset({1, 2}),
 frozenset({1, 3})]

In [10]:
suppData

{frozenset({1}): 0.5,
 frozenset({3}): 0.75,
 frozenset({4}): 0.25,
 frozenset({2}): 0.75,
 frozenset({5}): 0.75,
 frozenset({1, 3}): 0.5,
 frozenset({2, 5}): 0.75,
 frozenset({3, 5}): 0.5,
 frozenset({2, 3}): 0.5,
 frozenset({1, 5}): 0.25,
 frozenset({1, 2}): 0.25,
 frozenset({2, 3, 5}): 0.5}

In [11]:
apriori(dataSet, 0.7)

([[frozenset({5}), frozenset({2}), frozenset({3})], [frozenset({2, 5})], []],
 {frozenset({1}): 0.5,
  frozenset({3}): 0.75,
  frozenset({4}): 0.25,
  frozenset({2}): 0.75,
  frozenset({5}): 0.75,
  frozenset({2, 5}): 0.75,
  frozenset({3, 5}): 0.5,
  frozenset({2, 3}): 0.5})

## 挖掘关联规则

In [12]:
def generateRules(L, supportData, minConf=0.7):
    '''
    输入参数是apriori函数的输出结果
    生成包含可信度的规则列表
    '''
    bigRuleList = []
    for i in range(1, len(L)):  #只能从元素数大于等于2的项集进行规则构建
        for freqSet in L[i]:  #对内层列表遍历得到每个频繁项集
            H1 = [frozenset([item]) for item in freqSet]  #包含单个元素集合的列表,必须要用函数frozenset([]),还不能去掉里面那个中括号
            if(i > 1):  #频繁项集的元素个数超过2，进一步进行合并
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:  #两个元素，直接计算可信度
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList

def calcConf(freqSet, H, supportData, br1, minConf=0.7):
    prunedH = []
    for conseq in H:
        conf = supportData[freqSet] / supportData[freqSet - conseq]
        if conf >= minConf:
            print(freqSet - conseq, "-->", conseq, "conf:", conf)
            br1.append((freqSet - conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH

def rulesFromConseq(freqSet, H, supportData, br1, minConf=0.7):  #这个地方没有返回
    m = len(H[0])
    if (len(freqSet) > (m+1)):
        Hmp1 = aprioriGen(H, m+1)
        Hmp1 = calcConf(freqSet, Hmp1, supportData, br1, minConf)
        if (len(Hmp1) > 1):
            rulesFromConseq(freqSet, Hmp1, supportData, br1, minConf)

In [13]:
L, suppData = apriori(dataSet, minSupport = 0.5)
rules = generateRules(L, suppData, minConf=0.7)
rules

frozenset({5}) --> frozenset({2}) conf: 1.0
frozenset({2}) --> frozenset({5}) conf: 1.0
frozenset({1}) --> frozenset({3}) conf: 1.0


[(frozenset({5}), frozenset({2}), 1.0),
 (frozenset({2}), frozenset({5}), 1.0),
 (frozenset({1}), frozenset({3}), 1.0)]

In [14]:
rules = generateRules(L, suppData, minConf=0.5)
rules

frozenset({3}) --> frozenset({2}) conf: 0.6666666666666666
frozenset({2}) --> frozenset({3}) conf: 0.6666666666666666
frozenset({5}) --> frozenset({3}) conf: 0.6666666666666666
frozenset({3}) --> frozenset({5}) conf: 0.6666666666666666
frozenset({5}) --> frozenset({2}) conf: 1.0
frozenset({2}) --> frozenset({5}) conf: 1.0
frozenset({3}) --> frozenset({1}) conf: 0.6666666666666666
frozenset({1}) --> frozenset({3}) conf: 1.0
frozenset({5}) --> frozenset({2, 3}) conf: 0.6666666666666666
frozenset({3}) --> frozenset({2, 5}) conf: 0.6666666666666666
frozenset({2}) --> frozenset({3, 5}) conf: 0.6666666666666666


[(frozenset({3}), frozenset({2}), 0.6666666666666666),
 (frozenset({2}), frozenset({3}), 0.6666666666666666),
 (frozenset({5}), frozenset({3}), 0.6666666666666666),
 (frozenset({3}), frozenset({5}), 0.6666666666666666),
 (frozenset({5}), frozenset({2}), 1.0),
 (frozenset({2}), frozenset({5}), 1.0),
 (frozenset({3}), frozenset({1}), 0.6666666666666666),
 (frozenset({1}), frozenset({3}), 1.0),
 (frozenset({5}), frozenset({2, 3}), 0.6666666666666666),
 (frozenset({3}), frozenset({2, 5}), 0.6666666666666666),
 (frozenset({2}), frozenset({3, 5}), 0.6666666666666666)]

# 发现毒蘑菇的相似特征

###  第一个特征为是否有毒，有毒值为2，可食用值为1.
###  寻找包含特征值为2的频繁项集

In [15]:
mushDatSet = [line.split() for line in open('mushroom.dat').readlines()]

In [24]:
print(len(mushDatSet))
print(mushDatSet)

8124
[['1', '3', '9', '13', '23', '25', '34', '36', '38', '40', '52', '54', '59', '63', '67', '76', '85', '86', '90', '93', '98', '107', '113'], ['2', '3', '9', '14', '23', '26', '34', '36', '39', '40', '52', '55', '59', '63', '67', '76', '85', '86', '90', '93', '99', '108', '114'], ['2', '4', '9', '15', '23', '27', '34', '36', '39', '41', '52', '55', '59', '63', '67', '76', '85', '86', '90', '93', '99', '108', '115'], ['1', '3', '10', '15', '23', '25', '34', '36', '38', '41', '52', '54', '59', '63', '67', '76', '85', '86', '90', '93', '98', '107', '113'], ['2', '3', '9', '16', '24', '28', '34', '37', '39', '40', '53', '54', '59', '63', '67', '76', '85', '86', '90', '94', '99', '109', '114'], ['2', '3', '10', '14', '23', '26', '34', '36', '39', '41', '52', '55', '59', '63', '67', '76', '85', '86', '90', '93', '98', '108', '114'], ['2', '4', '9', '15', '23', '26', '34', '36', '39', '42', '52', '55', '59', '63', '67', '76', '85', '86', '90', '93', '98', '108', '115'], ['2', '4', '10', '1

In [17]:
L, suppData = apriori(mushDatSet, minSupport=0.4)

In [18]:
for item in L[1]:
    if item.intersection('2'):
        print(item)

frozenset({'2', '28'})
frozenset({'34', '2'})
frozenset({'59', '2'})
frozenset({'63', '2'})
frozenset({'85', '2'})
frozenset({'86', '2'})
frozenset({'90', '2'})
frozenset({'39', '2'})


In [19]:
for item in L[3]:
    if item.intersection('2'):
        print(item)

frozenset({'85', '34', '59', '2'})
frozenset({'86', '34', '59', '2'})
frozenset({'86', '85', '34', '2'})
frozenset({'86', '85', '59', '2'})
frozenset({'86', '90', '34', '2'})
frozenset({'86', '90', '85', '2'})
frozenset({'90', '85', '34', '2'})
frozenset({'90', '85', '59', '2'})
frozenset({'39', '85', '34', '2'})
frozenset({'39', '85', '59', '2'})
frozenset({'86', '39', '34', '2'})
frozenset({'86', '2', '39', '85'})
frozenset({'90', '39', '85', '2'})


# 风暴潮实例

In [27]:
data = [line.split() for line in open('fengbaochao.txt').readlines()]
print(len(data))
print(data)
C1 = createC1(data)
print(C1)

15
[['980', '25', '460', '120', '1.61', '35', '12', '0.315', '0.1394', '0.2', '0.75', '2', '3.46', '101', '2.4352', '7.3', '466.82', '2253.7'], ['950', '45', '600', '200', '2.41', '74', '14', '7.683', '24.7353', '35.5', '131.64', '7', '613.5', '18015', '2.7703', '6.6', '473.08', '2253.7'], ['990', '23', '430', '100', '1.96', '22', '32', '1.541', '3.961', '7.12', '26.4', '0', '7.71', '3613', '3.6676', '5.3', '488.63', '2253.7'], ['970', '33', '350', '80', '2.43', '88', '20', '6.61', '6.966', '32.65', '165', '7', '56.53', '16579', '3.3437', '9.7', '295.65', '3023.6'], ['998', '18', '150', '0', '1.52', '2', '0', '0.78', '0.371', '0.533', '5.3', '0', '9.211', '0', '2.4352', '7.3', '466.82', '2253.7'], ['1000', '16', '160', '0', '1.63', '15', '0', '0.039', '0.125', '0.18', '0.67', '0', '3.11', '0', '3.7638', '5.5', '550.53', '4314.1'], ['935', '51', '460', '220', '1.43', '31', '0', '0.065', '0.089', '0.3', '1.11', '0', '2.34', '0', '4.1405', '5.1', '494.03', '2253.7'], ['950', '45', '500', 

In [42]:
L, suppData = apriori(data, minSupport=0.25)

In [43]:
L

[[frozenset({'3023.6'}),
  frozenset({'0'}),
  frozenset({'100'}),
  frozenset({'2253.7'})],
 []]

In [44]:
suppData

{frozenset({'980'}): 0.13333333333333333,
 frozenset({'25'}): 0.13333333333333333,
 frozenset({'460'}): 0.13333333333333333,
 frozenset({'120'}): 0.13333333333333333,
 frozenset({'1.61'}): 0.06666666666666667,
 frozenset({'35'}): 0.2,
 frozenset({'12'}): 0.13333333333333333,
 frozenset({'0.315'}): 0.06666666666666667,
 frozenset({'0.1394'}): 0.06666666666666667,
 frozenset({'0.2'}): 0.06666666666666667,
 frozenset({'0.75'}): 0.06666666666666667,
 frozenset({'2'}): 0.2,
 frozenset({'3.46'}): 0.06666666666666667,
 frozenset({'101'}): 0.06666666666666667,
 frozenset({'2.4352'}): 0.2,
 frozenset({'7.3'}): 0.2,
 frozenset({'466.82'}): 0.2,
 frozenset({'2253.7'}): 0.4,
 frozenset({'950'}): 0.13333333333333333,
 frozenset({'45'}): 0.2,
 frozenset({'600'}): 0.06666666666666667,
 frozenset({'200'}): 0.06666666666666667,
 frozenset({'2.41'}): 0.06666666666666667,
 frozenset({'74'}): 0.06666666666666667,
 frozenset({'14'}): 0.06666666666666667,
 frozenset({'7.683'}): 0.06666666666666667,
 frozens